# Gillespie's Stochastic Simulation Algorithm

As we saw at lesson the SSA algorithm is used to simulate stochastic processes like reactions, we saw that when a system is not stochastic the reaction comes to an equilibrium and the system get's to a steady state what will happen with a stochastic system?


## Foundations of SSA

Given $m$ reactions $R_1, R_2, \dots, R_m$ and some molecules (or genes as we saw at lesson) $S_1, S_2, \dots, S_n$ we can write the notion of state as a vector on $n$ states $(X_1, X_2, \dots, X_n)$.

- When will the next reaction occur?
- Which reaction will occur?

So now we introduce the notion of propensity function $\alpha_i$ for each reaction $R_i$.
And we defien a probability distribution $P(\tau, \mu)$, where $\tau$ is the time interval we're considering from $t$ to $t + \tau$ and $R_\mu$ is the reaction that will occur.

We can call $c_\mu$ our constant that determines the rate of reactions. Let's define $\alpha_\mu$ as the $\color{red}{\text{propensity}}$ of the system, that is the sum of all the propensities of the reactions.

$$
\alpha_\mu = h_\mu c_\mu
$$

where $h_\mu$ is the number of molecules that can react : *Reactants*

So $R_\mu$ will be $l_1S_1 + ... + l_nS_n \rightarrow^c p_1S_1 + ... + p_nS_n$
Such that:

$$
h_\mu = \prod_{i=1}^n \binom{X_i}{l_i}
$$


## Where propensity is used?

So given a reaction set with certain propensities, we will use this as a parameter of an exponential distribution modelling the time between events occurrences. Remember that the exponential distribution is defined as:


CDF:

$$
F(x;\lambda) = 1 - e^{-\lambda x}
$$

And PDF:

$$
f(x) =
\begin{cases}
\lambda e^{-{\lambda}x} \;\;\;  x \geq 0 \\
0 \;\;\; otherwise
\end{cases}

$$

### An important point

What we must remember is that this distribution is memoryless and if we consider a set of RV, their min $(X_{set})$ is exponentially distributed

- Memoryless: $F(x;\lambda) = F(x + y;\lambda) = F(y;\lambda)$
- Min($X_{set}$) is exponentially distributed: $X = min(X_1, X_2, ..., X_n)$, then $X$ is exponentially distributed.

## The algorithm

We represent the state of simulation with a vector the multiset of molecules in the system: $[X_1,...,X_n]$. A real variable $t$ that is our time initially at 0.

The algorithm iterates as follows until $t$ reaches a certain value $t_{max}$:

1. The time $t+\tau$ so the time in which the next reaction will occur is given by the exponential distribution with parameter $\alpha_0 = \sum_{i=1}^m \alpha_i$.
2. The reaction $R_\mu$ that will occur at the time $t+\tau$ is chosen randomly with probability $\frac{\alpha_\mu}{\sum \alpha_i}$. This is $\frac{\alpha_\mu}{\alpha_0}$. This is the probability of the reaction to occur. 

At each step $t$ is incremented by $\tau$ and the state of the system is updated according to the reaction $R_\mu$ that occurred.

In [1]:

# In this program we will see how a negative feedback loop can be modeled using the Gillespie algorithm

# Our negative feedback loop will be formed by Genes (G1,G2,G3) and Proteins (P1,P2,P3) 
# such that proteins are inhibitory to the genes and genes are inhibitory to the proteins


import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# We will define the following parameters: these first all all our reaction with rate

# g1 rate of occurence g1,p1 is 10
# g2 rate of occurrence g2,p2 is 10000
# g3 rate of occurrence g3,p3 is 10
g1_production_rate = 10
g2_production_rate = 10000
g3_production_rate = 10

# when p1 binds to g2 there is a two way reaction :
# g2 + p1 -> g2p1 (molecule) the rate of occurrence is 10, and g2p1 -> g2 + p1 is 2
p1_binding_to_g2_rate = 10
p1_unbinding_from_g2_rate = 2
# when p2 binds to g3 there is a two way reaction :
# g3 + p2 -> g3p2 (molecule) the rate of occurrence is 0.1, and g3p2 -> g3 + p2 is 20
p2_binding_to_g3_rate = 0.1
p2_unbinding_from_g3_rate = 20
# when p3 binds to g1 there is a two way reaction :
# g1 + p3 -> g1p3 (molecule) the rate of occurrence is 10, and g1p3 -> g1 + p3 is 20
p3_binding_to_g1_rate = 10
p3_unbinding_from_g1_rate = 20

# Finally the rate of decay (protein denaturation we can say) of proteins is respectively :
# p1 rate of decay is 1
p1_decay_rate = 1
# p2 rate of decay is 100
p2_decay_rate = 100
# p3 rate of decay is 1
p3_decay_rate = 1


# Initial conditions
p1 = 0
p2 = 0
p3 = 0

g1 = 1
g2 = 1
g3 = 1

#So initial state is G = (1,1,1) and P = (0,0,0), t=0

# Let's try a smaller reaction first

We have Y1 and Y2 

Y1 with a rate of 10 becomes 2Y1

Y1+Y2 witha a rate of 0.01 becomes 2Y2

and Y2 with a rate of 10 becomes Z

3 reactions, 3 molecules.

In [10]:
y1_production_rate = 10 # Rate of occurrence of this reaction
y2y1_production_rate = 0.01 # Rate of occurrence of this reaction
y2z_production_rate = 10 # Rate of occurrence of this reaction

initial_state_simple = [1000,1000,0]

number_of_reactions = 3

In [15]:
def gillespies_SSA(initial_state, time_limit):
    
    propensities = []
    results = []


    for i in range(0, time_limit):

        #compute the propensities for each reaction
        propensities.append(initial_state[0]*y1_production_rate)
        propensities.append(initial_state[1]*y2y1_production_rate)
        propensities.append(initial_state[2]*y2z_production_rate)   
        
        # total propensity
        total_propensity = sum(propensities)
        
        # generate tau exponentially distributed, where the average is 1/total_propensity
        tau = np.random.exponential(1/total_propensity)
        
        #Choose the reaction to occur with respect to the propensities
        # As impementation detail in the slides: generate N a uniformly distributed number in [0 to a0],
        # Choosing reaction R_\mu with probability a_\mu / a0.
       
        
        print("Reaction chosen is ", reaction)
        
        # update the state
        if reaction == 0:
            initial_state[0] = 2*initial_state[0]
        elif reaction == 1:
            # y1+y2 -> 2y2
            initial_state[1] = 2*initial_state[1]
            # and y1,y2 are consumed
            initial_state[0] = initial_state[0] - 1
            initial_state[1] = initial_state[1] - 1
        elif reaction == 2:
            # y2 -> z
            initial_state[2] = initial_state[2] + 1
            # and y2 is consumed
            initial_state[1] = initial_state[1] - 1
            
        # update the results appending the new state
        results.append(initial_state)
    
    # gather the quantities of initial_state and return them to plot
    return results
           
def plot_results(results):
    
    plt.plot(results[:,0], label='y1')
    plt.plot(results[:,1], label='y2')
    plt.plot(results[:,2], label='y3')
    plt.legend()
    plt.show()



In [16]:
#gillespies_SSA(initial_state=(p1,p2,p3,g1,g2,g3), time_limit=100)
results = gillespies_SSA(initial_state=initial_state_simple, time_limit=10)

plot_results(results)

Reaction chosen is  [0]


ValueError: probabilities do not sum to 1